In [2]:
import fullcontrol as fc
from pint import UnitRegistry
u = UnitRegistry()

In [3]:
#http://www.aardvarkmastering.com/riaa.htm

In [7]:
NUM_POINTS = 1000
OUTER_RADIUS = 5.75 #inch
INNER_RADIUS = 2.25 #inch 
ROTATION_COUNT = 5


steps = []

for i in range(NUM_POINTS):
    step = fc.polar_to_point(centre=fc.Point(x=0, y=0, z=2),
                              radius=(OUTER_RADIUS-INNER_RADIUS)*(i/NUM_POINTS)+INNER_RADIUS, 
                              angle=i*(ROTATION_COUNT*2*3.14159/NUM_POINTS))
    step.color = [1, 0, 0]
    steps.append(fc.ExtrusionGeometry(width=i/NUM_POINTS))
    steps.append(step)
fc.transform(steps, 'plot', fc.PlotControls(color_type='manual', neat_for_publishing=True, zoom=0.7))

    - use fc.transform(..., controls=fc.PlotControls(style='tube') to disable this message or style='line' for a simpler line preview

fc.transform guidance tips are being written to screen if any potential issues are found - hide tips with fc.transform(..., show_tips=False)
tip: set initial `extrusion_width` and `extrusion_height` in the initialization_data to ensure the preview is correct:
   - `fc.transform(..., controls=fc.PlotControls(initialization_data={'extrusion_width': EW, 'extrusion_height': EH}))`



In [5]:
RECORD_SPEED = 33*u.rpm
((ROTATION_COUNT*u.revolutions)/RECORD_SPEED).to_base_units()

<Quantity(9.09090909, 'second')>